# Table of Contents
* [Submitting HITs](#Submitting-HITs)
	* [Building URLs for images on s3](#Building-URLs-for-images-on-s3)
	* [submitting HITs in groups](#submitting-HITs-in-groups)
* [Reviewing HITs](#Reviewing-HITs)


In [65]:
import boto.mturk.connection as tc
import boto.mturk.question as tq
from keysTkingdom import mturk_ai2
import pickle

# Submitting HITs

## Building URLs for images on s3

In [60]:
def load_book_info():
    with open('breakdowns.pkl', 'rb') as f:
        book_breakdowns = pickle.load(f, encoding='latin1')

    with open('pdfs/page_ranges.csv') as f:
        ranges = f.readlines()
    range_lookup = {line.split(' ')[0]:[int(num) for num in line.strip().split(' ')[1:]] for line in ranges}
    return book_breakdowns, range_lookup

def form_hit_url(book_name, page_n):
    book_name_no_ext = book_name.replace('.pdf', '_')
    baseurl = 'https://s3-us-west-2.amazonaws.com/ai2-vision-turk-data/textbook-annotation-test/build/index.html'
    full_url = baseurl + '?url={}{}.jpeg&id={}'.format(book_name_no_ext, page_n, page_n)
    return full_url

def make_book_group_urls(book_group):
    group_urls = []
    def get_start_end(start, end):
        return start, end
    
    for tb in book_groups[book_group]:
        start, end = get_start_end(*ranges[tb])
        for page_n in range(start, end):
            group_urls.append(form_hit_url(tb, page_n))
    return group_urls

In [57]:
book_groups,ranges = load_book_info()

In [68]:
daily_sci_urls = make_book_group_urls('daily_sci')
spectrum_sci_urls = make_book_group_urls('spectrum_sci')

In [62]:
daily_sci_urls[98:100]

## submitting HITs in groups

In [61]:
def creat_single_hit(url):
    """
    creates a single HIT from a provided url
    """
    title = "Annotate Science Textbook"
    description = "Choose which category a text entry best belongs to"
    keywords = ['image', 'science']
    frame_height = 1000 # the height of the iframe holding the external hit
    amount = .05

    questionform = tq.ExternalQuestion(url, frame_height)

    create_hit_result = mturk.create_hit(
        title = title,
        description = description,
        keywords = keywords,
        question = questionform,
        reward = boto.mturk.price.Price(amount=amount),
        max_assignments=3
    )

In [66]:
sandbox_host = 'mechanicalturk.sandbox.amazonaws.com' 
mturk = tc.MTurkConnection(
    aws_access_key_id = mturk_ai2.access_key,
    aws_secret_access_key = mturk_ai2.access_secret_key,
    host = sandbox_host,
    debug = 1 # debug = 2 prints out all requests.
)

# Reviewing HITs

In [63]:
# batch_results_df = pd.read_csv(data_dir+results_csv)
# print(batch_results_df.shape)
# batch_results_df.head(2)

In [64]:
# grouped_results_df = batch_results_df.groupby('Input.image_url')
# for image_response in grouped_results_df:
#     print(image_response[1]['Answer.NumberOfItems'])